# Stock Price Prediction with LSTM
This notebook reproduces the script `stock_price_prediction.py` with sectioned execution.

## 1. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
import tensorflow as tf

# User-configurable parameters
TICKER = 'PLTR'  # Change ticker as needed
LOOKBACK = 60
EPOCHS = 50
BATCH_SIZE = 32
VAL_SPLIT = 0.1

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
print('Libraries imported and seeds set.')

## 2. Download Stock Data

In [ ]:
df = yf.download(TICKER, period='5y')
close_prices = df[['Close']]
print(f'Downloaded {df.shape[0]} rows of data for {TICKER}.')
close_prices.head()

## 3. Data Visualization

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(close_prices.index, close_prices['Close'], label='Close Price')
plt.title(f'Historical Closing Prices for {TICKER}')
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')
plt.legend()
plt.show()
print('Displayed historical closing prices.')

## 4. Data Preprocessing

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(close_prices.values)
training_data_len = int(np.ceil(len(scaled_data) * 0.8))
train_data = scaled_data[:training_data_len]
test_data = scaled_data[training_data_len-LOOKBACK:]

def create_dataset(dataset, lookback):
    X, y = [], []
    for i in range(lookback, len(dataset)):
        X.append(dataset[i-lookback:i,0])
        y.append(dataset[i,0])
    return np.array(X), np.array(y)

X_train, y_train = create_dataset(train_data, LOOKBACK)
X_test, y_test = create_dataset(test_data, LOOKBACK)
print(f'Training sequences: {X_train.shape}, Testing sequences: {X_test.shape}')

## 5. Reshape Data for LSTM

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
print(f'Reshaped X_train to {X_train.shape} and X_test to {X_test.shape}.')

## 6. Build the LSTM Model

In [ ]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(LOOKBACK,1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

## 7. Train the Model

In [ ]:
history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=VAL_SPLIT,
    verbose=1
)
plt.figure(figsize=(10,4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss During Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
print('Training complete.')

## 8. Make Predictions

In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
actual_prices = scaler.inverse_transform(y_test.reshape(-1,1))
print('First 5 predictions vs actuals:')
print(np.hstack((predictions[:5], actual_prices[:5])))

## 9. Evaluate Model Performance

In [ ]:
rmse = np.sqrt(mean_squared_error(actual_prices, predictions))
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')

## 10. Visualize Results

In [ ]:
train = close_prices[:training_data_len]
valid = close_prices[training_data_len:]
valid['Predictions'] = predictions
plt.figure(figsize=(14,5))
plt.plot(train.index, train['Close'], label='Training Data')
plt.plot(valid.index, valid['Close'], label='Actual Prices')
plt.plot(valid.index, valid['Predictions'], label='Predicted Prices')
plt.title(f'{TICKER} Price Prediction')
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')
plt.legend()
plt.show()
print('Displayed prediction results.')

## 11. Save the Model

In [ ]:
model.save('stock_price_model.h5')
print('Model saved to stock_price_model.h5')

## 12. Optional: Predict Future Prices

In [ ]:
future_predictions = []
last_sequence = scaled_data[-LOOKBACK:]
current_sequence = last_sequence.reshape(1, LOOKBACK, 1)
for _ in range(30):
    next_pred = model.predict(current_sequence)[0,0]
    future_predictions.append(next_pred)
    last_sequence = np.append(last_sequence, next_pred)[-LOOKBACK:]
    current_sequence = last_sequence.reshape(1, LOOKBACK, 1)
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1,1))
last_date = close_prices.index[-1]
future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=30)
plt.figure(figsize=(14,5))
plt.plot(close_prices.index, close_prices['Close'], label='Historical Prices')
plt.plot(future_dates, future_predictions, label='30-Day Forecast')
plt.title(f'{TICKER} Price Forecast')
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')
plt.legend()
plt.show()
print('Displayed future price forecast.')